In [1]:
import os
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Subset

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
print('Trainset size:', len(trainset))
print('Testset size:', len(testset))

100%|██████████| 9912422/9912422 [00:00<00:00, 80097003.36it/s]


Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 5366635.38it/s]


Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 18850451.63it/s]


Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 13165534.74it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw

Trainset size: 60000
Testset size: 10000


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
model = Net()
print(model)       

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "gpu")

In [5]:
def get_teacher_preds(num_teachers, num_examples, epochs): 
    dirName = "teachers " + str(num_teachers)
    if not os.path.exists(dirName):
        os.mkdir(dirName)
        
    os.chdir(dirName) 
    
    for i in range(num_teachers):
        train_idx = list(range(i * num_examples, (i+1) * num_examples))
        train = Subset(trainset, train_idx)
        trainloader = torch.utils.data.DataLoader(train, batch_size=64, num_workers=0)      
       
        model = Net()
        criterion = nn.NLLLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)        
        model.to(device)
        
        for e in range(epochs):
            running_loss = 0
            for images, labels in trainloader:                 
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()        
                output = model(images)
                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()        
                running_loss += loss.item() 
                
        print("Teacher_" + str(i+1) + " training loss:", running_loss/len(trainloader))  
        
        student_idx = list(range(0, 6000))      
        student_data = Subset(testset, student_idx)  
        student_loader = torch.utils.data.DataLoader(student_data, batch_size=64, num_workers=0)
        
        model.to(device) 
        model.eval()
        
        outputs = torch.zeros(0, dtype=torch.long).to(device)
        for images, labels in student_loader:             
            images, labels = images.to(device), labels.to(device)                             
            output = model.forward(images)
            pred = torch.argmax(torch.exp(output), dim=1)
            outputs = torch.cat((outputs, pred))            
    
        file_name = 'pred_' + str(i+1) + '.pt'  
        torch.save(outputs, file_name)          

In [6]:
num_teachers = 50
num_examples = len(trainset)//num_teachers
epochs = 10

get_teacher_preds(num_teachers, num_examples, epochs)

<ipython-input-3-34973861982a>:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Teacher_1 training loss: 0.5753144527736463
Teacher_2 training loss: 0.5145812269888426
Teacher_3 training loss: 0.5327097773551941
Teacher_4 training loss: 0.4974764209044607
Teacher_5 training loss: 0.545142788636057
Teacher_6 training loss: 0.470995622245889
Teacher_7 training loss: 0.6517519448932848
Teacher_8 training loss: 0.6131471944482703
Teacher_9 training loss: 0.5295835341277876
Teacher_10 training loss: 0.5776217877864838
Teacher_11 training loss: 0.5384372504133927
Teacher_12 training loss: 0.6522518615973624
Teacher_13 training loss: 0.6246600135376579
Teacher_14 training loss: 0.5193701511935184
Teacher_15 training loss: 0.6368799099796697
Teacher_16 training loss: 0.44207262522295904
Teacher_17 training loss: 0.5327953774678079
Teacher_18 training loss: 0.4821277417634663
Teacher_19 training loss: 0.5971845545266804
Teacher_20 training loss: 0.5016139337891027
Teacher_21 training loss: 0.5990494913176486
Teacher_22 training loss: 0.5433270868502165
Teacher_23 training 

In [7]:
!pwd

/content/teachers 50


In [8]:
num_teachers = 50

preds = []
for i in range(num_teachers): 
    file_name = 'pred_' + str(i+1) + '.pt'
    pred = torch.load(file_name).cpu().numpy()
    preds.append(pred)    
    teacher_preds = np.vstack((preds))
print(teacher_preds.shape)

(50, 6000)


In [9]:
def aggragate_teacher_preds(epsilon):  
    
    labels = np.array([]).astype(int)
    for pred in np.transpose(teacher_preds):   
        label_counts = np.bincount(pred, minlength=10)    
        beta = 1 / epsilon

        for i in range(len(label_counts)):
            label_counts[i] += np.random.exponential(1/beta,size)
        
        new_label = np.argmax(label_counts)   
        labels = np.append(labels, new_label) 
        labels =  torch.from_numpy(labels)      
   
    return labels

In [10]:
num_teachers, num_examples, num_labels = (50, 6000, 10)
diff_priv_labels = aggragate_teacher_preds(0.2)  

In [11]:
testset.data[:6000] = torch.FloatTensor(testset.data.clone().detach().numpy()[:6000])
testset.targets[:6000] = diff_priv_labels

student_train = Subset(testset, list(range(6000)))
student_trainloader = torch.utils.data.DataLoader(student_train, batch_size=64, num_workers=0)

model_2 = Net()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model_2.parameters(), lr=0.001)
model_2.to(device)
epochs = 15

for e in range(epochs): 
    train_loss = 0.0    
    for images, labels in student_trainloader:          
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()        
        output = model_2(images)        
        loss = criterion(output, labels)        
        loss.backward()       
        optimizer.step()        
        train_loss += loss.item()
            
    print("Epoch: {}/{}   ".format(e+1, epochs),
                      "Training Loss: {:.5f}   ".format(train_loss/len(student_trainloader)))  

<ipython-input-3-34973861982a>:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Epoch: 1/15    Training Loss: 1.55636   
Epoch: 2/15    Training Loss: 0.58830   
Epoch: 3/15    Training Loss: 0.41598   
Epoch: 4/15    Training Loss: 0.35026   
Epoch: 5/15    Training Loss: 0.32910   
Epoch: 6/15    Training Loss: 0.29818   
Epoch: 7/15    Training Loss: 0.29510   
Epoch: 8/15    Training Loss: 0.28079   
Epoch: 9/15    Training Loss: 0.25652   
Epoch: 10/15    Training Loss: 0.25523   
Epoch: 11/15    Training Loss: 0.25206   
Epoch: 12/15    Training Loss: 0.23656   
Epoch: 13/15    Training Loss: 0.22423   
Epoch: 14/15    Training Loss: 0.22195   
Epoch: 15/15    Training Loss: 0.22013   


In [12]:
student_test = Subset(testset, list(range(6000, 10000)))  
student_testloader = torch.utils.data.DataLoader(student_test, batch_size=64, num_workers=0)

test_loss = 0
accuracy = 0
model_2.to(device)
model_2.eval()

with torch.no_grad(): 
    for images, labels in student_testloader:              
        images, labels = images.to(device), labels.to(device)
        output = model_2(images)
        test_loss += criterion(output, labels).item()        
               
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim=1)        
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))

print("Test Loss: {:.5f}   ".format(test_loss/len(student_testloader)),
      "Test Accuracy: {:.5f}".format(accuracy/len(student_testloader)))

<ipython-input-3-34973861982a>:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Test Loss: 0.18312    Test Accuracy: 0.95263
